In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

# 1. Cargar los datos
train_data = pd.read_csv('train_processed.csv')
test_data = pd.read_csv('test_processed.csv')

In [ ]:
# 2. Análisis exploratorio de datos
print("Información del conjunto de datos de entrenamiento:")
print(f"Número de filas: {train_data.shape[0]}")
print(f"Número de columnas: {train_data.shape[1]}")
print("\nPrimeras filas del conjunto de datos:")
print(train_data.head())

# Verificar valores nulos
print("\nValores nulos en el conjunto de entrenamiento:")
print(train_data.isnull().sum())

# Estadísticas descriptivas de las características numéricas
print("\nEstadísticas descriptivas:")
print(train_data.describe())

# Variables objetivo
print("\nEstadísticas de la variable objetivo 'prezo_euros':")
print(train_data['prezo_euros'].describe())

# Visualizar la distribución del precio
plt.figure(figsize=(10, 6))
sns.histplot(train_data['prezo_euros'], kde=True)
plt.title('Distribución de precios de viviendas')
plt.xlabel('Precio (euros)')
plt.ylabel('Frecuencia')
plt.savefig('distribucion_precios.png')
plt.close()

# También visualizamos el logaritmo del precio (que suele ser más normal)
plt.figure(figsize=(10, 6))
sns.histplot(train_data['log_prezo'], kde=True)
plt.title('Distribución del logaritmo de precios de viviendas')
plt.xlabel('Log(Precio)')
plt.ylabel('Frecuencia')
plt.savefig('distribucion_log_precios.png')
plt.close()




Información del conjunto de datos de entrenamiento:
Número de filas: 20000
Número de columnas: 55

Primeras filas del conjunto de datos:
      id  superficie_interior_m2  superficie_exterior_m2  numero_habitacions  \
0  25521               -1.214878               -0.600905                   1   
1   4843               -0.590517               -0.493874                   2   
2  27734                1.723563               -0.371180                   1   
3  22142                0.415219               -0.533578                   4   
4  14748                1.203970                2.040511                   1   

   numero_banos  ano_construccion  lonxitude  latitude  \
0             2              1947      -8.17     43.20   
1             2              1977      -7.23     43.60   
2             1              1996      -8.40     42.25   
3             2              1996      -6.81     43.15   
4             1              1990      -8.76     42.92   

   temperatura_media_mes_construc

In [13]:
# 3. Preparación de datos para modelado
# Identificar características y variable objetivo
# Excluimos 'id', 'prezo_euros', 'log_prezo' y columnas duplicadas al final del dataset
target_column = 'prezo_euros'
log_target_column = 'log_prezo'

# Eliminar columnas duplicadas y el ID
all_columns = train_data.columns.tolist()
features = [col for col in all_columns if col not in ['id', 'prezo_euros', 'log_prezo', 'is_outlier']]

# Eliminamos columnas duplicadas que aparecen dos veces al final del dataset
unique_features = []
for feature in features:
    if feature not in unique_features:
        unique_features.append(feature)
    else:
        print(f"Columna duplicada encontrada: {feature}")

features = unique_features
print(f"\nNúmero de características utilizadas: {len(features)}")

# Separar features y target
X = train_data[features]
y = train_data[target_column]  # Predecimos el precio real

# Identificar las columnas categóricas
categorical_columns = X.select_dtypes(include=['object']).columns
print("Columnas categóricas:", categorical_columns)

label_encoder = LabelEncoder()
for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col])

y_log = train_data[log_target_column]  # También guardamos el log para probarlo

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
_, _, y_log_train, y_log_val = train_test_split(X, y_log, test_size=0.2, random_state=42)


Número de características utilizadas: 51
Columnas categóricas: Index(['tipo_edificacion', 'calidade_materiais', 'cor_favorita_propietario',
       'acceso_transporte_publico', 'orientacion', 'eficiencia_enerxetica'],
      dtype='object')


In [14]:
# 4. Definir función para evaluar modelos
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"Métricas para {model_name}:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.4f}")
    print("-" * 50)
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

In [15]:
# 5. Entrenar y evaluar modelos base
models = {}
results = {}

# Modelo 1: Random Forest
print("\nEntrenando Random Forest...")
rf_model = RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_val)
results['Random Forest'] = evaluate_model(y_val, rf_pred, "Random Forest")
models['Random Forest'] = rf_model

# Modelo 2: Gradient Boosting
print("\nEntrenando Gradient Boosting...")
gb_model = GradientBoostingRegressor(random_state=42, n_estimators=100)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_val)
results['Gradient Boosting'] = evaluate_model(y_val, gb_pred, "Gradient Boosting")
models['Gradient Boosting'] = gb_model

# Modelo 3: XGBoost
print("\nEntrenando XGBoost...")
xgb_model = XGBRegressor(random_state=42, n_estimators=100, n_jobs=-1)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_val)
results['XGBoost'] = evaluate_model(y_val, xgb_pred, "XGBoost")
models['XGBoost'] = xgb_model

# Modelo 4: MLP (Red Neuronal)
print("\nEntrenando MLP (Red Neuronal)...")
mlp_model = MLPRegressor(random_state=42, max_iter=500, hidden_layer_sizes=(100, 50), early_stopping=True)
mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_val)
results['MLP'] = evaluate_model(y_val, mlp_pred, "MLP (Red Neuronal)")
models['MLP'] = mlp_model

# Modelo 5: Ridge Regression
print("\nEntrenando Ridge Regression...")
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_val)
results['Ridge'] = evaluate_model(y_val, ridge_pred, "Ridge Regression")
models['Ridge'] = ridge_model



Entrenando Random Forest...
Métricas para Random Forest:
MSE: 2068847472.45
RMSE: 45484.58
MAE: 32662.67
R²: 0.9259
--------------------------------------------------

Entrenando Gradient Boosting...
Métricas para Gradient Boosting:
MSE: 1886817918.07
RMSE: 43437.52
MAE: 31023.61
R²: 0.9324
--------------------------------------------------

Entrenando XGBoost...
Métricas para XGBoost:
MSE: 2016695379.30
RMSE: 44907.63
MAE: 32034.54
R²: 0.9277
--------------------------------------------------

Entrenando MLP (Red Neuronal)...
Métricas para MLP (Red Neuronal):
MSE: 2664601883.16
RMSE: 51619.78
MAE: 33603.67
R²: 0.9045
--------------------------------------------------

Entrenando Ridge Regression...
Métricas para Ridge Regression:
MSE: 3078500588.76
RMSE: 55484.24
MAE: 38424.80
R²: 0.8897
--------------------------------------------------


In [16]:
# 6. Visualizar comparación de resultados
models_df = pd.DataFrame(results).T
models_df = models_df.sort_values('rmse')

plt.figure(figsize=(12, 6))
sns.barplot(x=models_df.index, y=models_df['rmse'])
plt.title('RMSE por Modelo')
plt.ylabel('RMSE')
plt.xlabel('Modelo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('comparacion_modelos_rmse.png')
plt.close()

plt.figure(figsize=(12, 6))
sns.barplot(x=models_df.index, y=models_df['r2'])
plt.title('R² por Modelo')
plt.ylabel('R²')
plt.xlabel('Modelo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('comparacion_modelos_r2.png')
plt.close()

In [17]:
# 7. Optimización de hiperparámetros para los mejores modelos
# Seleccionamos los tres mejores modelos para optimizar
best_models = list(models_df.iloc[:3].index)
print(f"\nLos mejores modelos son: {best_models}")

# Definimos los hiperparámetros a optimizar
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0]
    },
    'MLP': {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate_init': [0.001, 0.01]
    },
    'Ridge': {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    }
}

# Optimización de hiperparámetros para los mejores modelos
optimized_models = {}

for model_name in best_models:
    if model_name in param_grids:
        print(f"\nOptimizando hiperparámetros para {model_name}...")
        
        # Seleccionamos el modelo base correspondiente
        if model_name == 'Random Forest':
            base_model = RandomForestRegressor(random_state=42, n_jobs=-1)
        elif model_name == 'XGBoost':
            base_model = XGBRegressor(random_state=42, n_jobs=-1)
        elif model_name == 'Gradient Boosting':
            base_model = GradientBoostingRegressor(random_state=42)
        elif model_name == 'MLP':
            base_model = MLPRegressor(random_state=42, max_iter=500, early_stopping=True)
        elif model_name == 'Ridge':
            base_model = Ridge(random_state=42)
            
        # Configuramos la búsqueda en grid con validación cruzada
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grids[model_name],
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=0
        )
        
        # Ajustamos la búsqueda en grid
        grid_search.fit(X_train, y_train)
        
        # Guardamos el mejor modelo
        best_model = grid_search.best_estimator_
        optimized_models[model_name] = best_model
        
        # Evaluamos el modelo optimizado
        best_pred = best_model.predict(X_val)
        results[f'{model_name} (Optimizado)'] = evaluate_model(y_val, best_pred, f"{model_name} (Optimizado)")
        
        print(f"Mejores hiperparámetros para {model_name}:")
        print(grid_search.best_params_)



Los mejores modelos son: ['Gradient Boosting', 'XGBoost', 'Random Forest']

Optimizando hiperparámetros para Gradient Boosting...


KeyboardInterrupt: 

In [ ]:
# 8. Implementación de Stacking
# Definimos los modelos base y el meta-modelo
base_models = [
    (name, model) for name, model in optimized_models.items()
]

# Utilizamos Ridge como meta-modelo
meta_model = Ridge(random_state=42)

# Creamos el modelo de stacking
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print("\nEntrenando modelo de Stacking...")
stacking_model.fit(X_train, y_train)
stacking_pred = stacking_model.predict(X_val)
results['Stacking'] = evaluate_model(y_val, stacking_pred, "Stacking")


In [ ]:
# 9. Entrenamiento final con todos los datos y generación de predicciones
print("\nEntrenando modelo final con todos los datos...")
final_model = stacking_model
final_model.fit(X, y)

# Preparamos el conjunto de test
X_test = test_data[features]

# Realizamos la predicción en el conjunto de test
test_predictions = final_model.predict(X_test)

# Creamos el archivo de envío
submission = pd.DataFrame({
    'id': test_data['id'],
    'prezo_euros': test_predictions
})

# Guardamos el archivo de envío
submission.to_csv('submission.csv', index=False)
print("\nArchivo de envío 'submission.csv' generado correctamente.")

# 10. Visualizar importancia de características para algunos modelos
if 'Random Forest' in optimized_models:
    rf_model = optimized_models['Random Forest']
    
    # Importancia de características para Random Forest
    feature_importance_rf = pd.DataFrame({
        'Feature': features,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_rf.head(15))
    plt.title('Top 15 características más importantes (Random Forest)')
    plt.tight_layout()
    plt.savefig('feature_importance_rf.png')
    plt.close()
    
    print("\nCaracterísticas más importantes (Random Forest):")
    print(feature_importance_rf.head(10))

if 'XGBoost' in optimized_models:
    xgb_model = optimized_models['XGBoost']
    
    # Importancia de características para XGBoost
    feature_importance_xgb = pd.DataFrame({
        'Feature': features,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_xgb.head(15))
    plt.title('Top 15 características más importantes (XGBoost)')
    plt.tight_layout()
    plt.savefig('feature_importance_xgb.png')
    plt.close()
    
    print("\nCaracterísticas más importantes (XGBoost):")
    print(feature_importance_xgb.head(10))

print("\n¡Proceso completo! El modelo está listo para enviar a Kaggle.")